# Create Cluster: HDB
This notebook will create (start) an HDB cluster on a named database.

In [1]:
%%html
<style>
table {float:left}
</style>

## Setup

### Node Types
|Type|Mem (GB)|vCPUs|
|:---|---:|---:|
|kx.s.large|12|2|
|kx.s.xlarge|27|4|
|kx.s.2xlarge|54|8|
|kx.s.4xlarge|108|16|
|kx.s.8xlarge|216|32|
|kx.s.16xlarge|432|64|
|kx.s.32xlarge|864|128|


In [2]:
import os
import boto3
import json
import datetime

from managed_kx import *
from env_2 import *

# Managed KX Database and Cluster names to create
DB_NAME="welcomedb"

SEC_THREADS='4'
CLUSTER_NAME=f"HDB_{DB_NAME}"

# Cluster Settings
CODEBASE="code"
S3_CODE_PATH="code"

DB_PATHS = [ '/' ]

CAPACITY_CONFIG={ 'nodeCount': 3, 'nodeType': 'kx.s.2xlarge'}
DATABASE_CONFIG=[{ 'databaseName': DB_NAME,'cacheConfigurations': [{'dbPaths': DB_PATHS, 'cacheType': 'CACHE_1000' }] }]
CACHE_CONFIG=[{'type': 'CACHE_1000', 'size':1200}]

CODE_CONFIG={ 's3Bucket': S3_BUCKET, 's3Key': f'{S3_CODE_PATH}/{CODEBASE}.zip' }

INIT_SCRIPT=f'{CODEBASE}/init.q'
CMD_ARGS=[
    { 'key': 's', 'value': SEC_THREADS }, 
    { 'key': 'dbname', 'value': DB_NAME}, 
    { 'key': 'codebase', 'value': CODEBASE} 
]

In [3]:
# triggers credential get
session=None

try:
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS sessio: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


## Check Database

In [4]:
note_str = ""

c_set_list = []

try:
    c_set_list = client.list_kx_changesets(environmentId=ENV_ID, databaseName=DB_NAME)['kxChangesets']
except:
    note_str = "<<Could not get changesets>>"

print(100*"=")
print(f"Database: {DB_NAME}, Changesets: {len(c_set_list)} {note_str}")
print(100*"=")

# sort by create time
c_set_list = sorted(c_set_list, key=lambda d: d['createdTimestamp']) 

for c in c_set_list:
    c_set_id = c['changesetId']
    print(f"Changeset ({c['status']}): {c_set_id}: Created: {c['createdTimestamp']}")
    c_rqs = client.get_kx_changeset(environmentId=ENV_ID, databaseName=DB_NAME, changesetId=c_set_id)['changeRequests']

    chs_pdf = pd.DataFrame.from_dict(c_rqs).style.hide(axis='index')
    display(chs_pdf)

Database: welcomedb, Changesets: 1 
Changeset (COMPLETED): isRIOfpd2lyrudnX5paQQA: Created: 2023-06-06 11:24:58.428000+00:00


changeType,s3Path,dbPath
PUT,s3://kdb-demo-612841383594-kms/data/hdb/,/


## Create Cluster

In [5]:
resp = client.create_kx_cluster(
    environmentId=ENV_ID, 
    clusterName=CLUSTER_NAME,
    clusterType='HDB',
    releaseLabel = '1.0',
    capacityConfiguration=CAPACITY_CONFIG,
    databases=DATABASE_CONFIG,
    cacheStorageConfigurations=CACHE_CONFIG,
    clusterDescription="Created with create_cluster_HDB notebook",
    code=CODE_CONFIG,
    initializationScript=INIT_SCRIPT,
    commandLineArguments=CMD_ARGS,
    azMode=AZ_MODE,
    availabilityZoneId=AZ_ID,
    vpcConfiguration={ 
        'vpcId': VPC_ID,
        'securityGroupIds': SECURITY_GROUPS,
        'subnetIds': SUBNET_IDS,
        'ipAddressType': 'IP_V4' }
)

In [6]:
resp

{'ResponseMetadata': {'RequestId': '6047fec7-f921-4203-ac05-cf0451b21b78',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '1104',
   'connection': 'keep-alive',
   'date': 'Tue, 06 Jun 2023 11:32:14 GMT',
   'x-amzn-requestid': '6047fec7-f921-4203-ac05-cf0451b21b78',
   'x-amz-apigw-id': 'GGDhOHnToAMFuJg=',
   'x-amzn-trace-id': 'Root=1-647f193a-512446975580ccff4805d4c0',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 aef197034a978e986954f2826c90b090.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'IAD55-P1',
   'x-amz-cf-id': '3-QMC4ykphBRhaR2hJ5LX5xWHyT3mrdNQ1Txi2R7p-SUR2bVcbJt7g=='},
  'RetryAttempts': 0},
 'status': 'PENDING',
 'clusterName': 'HDB_welcomedb',
 'clusterType': 'HDB',
 'databases': [{'databaseName': 'welcomedb',
   'cacheConfigurations': [{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}],
   'changesetId': 'isRIOfpd2lyrudnX5paQQA'}],
 'cacheStorageConfigurations': [{'type': 'CACHE_1000', 'size': 1200}],
 'cluste

In [7]:
wait_for_cluster_status(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, show_wait=True)
print()
print("** DONE **")

Cluster: HDB_welcomedb status is PENDING, total wait 0:00:00, waiting 30 sec ...
Cluster: HDB_welcomedb status is CREATING, total wait 0:00:30, waiting 30 sec ...
Cluster: HDB_welcomedb status is CREATING, total wait 0:01:00, waiting 30 sec ...
Cluster: HDB_welcomedb status is CREATING, total wait 0:01:30, waiting 30 sec ...
Cluster: HDB_welcomedb status is CREATING, total wait 0:02:00, waiting 30 sec ...
Cluster: HDB_welcomedb status is CREATING, total wait 0:02:30, waiting 30 sec ...
Cluster: HDB_welcomedb status is CREATING, total wait 0:03:00, waiting 30 sec ...
Cluster: HDB_welcomedb status is CREATING, total wait 0:03:30, waiting 30 sec ...
Cluster: HDB_welcomedb status is CREATING, total wait 0:04:00, waiting 30 sec ...
Cluster: HDB_welcomedb status is CREATING, total wait 0:04:30, waiting 30 sec ...
Cluster: HDB_welcomedb status is CREATING, total wait 0:05:00, waiting 30 sec ...
Cluster: HDB_welcomedb status is CREATING, total wait 0:05:30, waiting 30 sec ...
Cluster: HDB_welc

In [ ]:
# Give permissions time to propogate after cluster creation....
time.sleep(60)

In [9]:
conn_str = get_kx_connection_string(client, environmentId=ENV_ID, clusterName=CLUSTER_NAME, userName=KDB_USERNAME, boto_session=session)
print(conn_str)

:tcps://vpce-059110fcb074ef975-uyzzkvgm.vpce-svc-099e64025966b40d1.us-east-1.vpce.amazonaws.com:443:bob:Host=vpce-059110fcb074ef975-uyzzkvgm.vpce-svc-099e64025966b40d1.us-east-1.vpce.amazonaws.com&Port=5000&User=bob&Action=finspace%3AConnectKxCluster&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEP3%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJHMEUCIQDk%2FWbQaFt94YuV%2BGV9OMrNc3uRQLc%2FH1MuGBWdDlEzqQIgb%2BGEJzmYHBxxABXdbR3NsVKDQzny8rzdZtmLa7Avzowq%2BAIIRhAAGgw2MTI4NDEzODM1OTQiDA%2F%2FljEC4Fa5qwBX7irVAjMODMSbjPuAOYavgu%2Ft142H0Yf47p1ftYb7dU1Lty4jDFWa9fqrsIIqYNRSR8nJ2GRkJBYWNMcxBEX5E7kfIZXiNQ6KM7hy41LwbpIDVo4So2fRDqbQj890uy5USBSJdkfFAPPKavGqs%2Bd4RJKjq%2FD3B2jlAad4rUXK8NmeXEBfPWgpC2Py45RIvjrjOUN7h9b5qRe99RBzsIWOxu1%2B%2FJL1RjLeCBk9uejOejbPtx0Hq3UQJN%2FihJsyaH1ACziG9LVBWM1Erkn06x6HFIZslFJ9T1ya5R8htG37U%2FMgHkKIaoesBWS1Wpr%2BpOLz5bBMAKtXlNcBZwv1A1uUUOKwLbWJfjzi65y%2BEGHEmjNxfDTUB5IoBNQOeoVcV1cTh6Wz3BXcBsuc8WtiLLEpLQjIZBiiKjBC1OmEDfTdcvwelRiQLtg1Ii4fPuwpX7rF99SkRQA1%2BLmqMM7c%2FKMGOr8B8chOD%2BWU%2

In [10]:
cdf = get_clusters(client, ENV_ID)

display(cdf)

clusterName,status,clusterType,capacityConfiguration,commandLineArguments,clusterDescription,lastModifiedTimestamp,createdTimestamp,databaseName,cacheConfigurations
HDB_TAQ_2021_2D,RUNNING,HDB,"{'nodeType': 'kx.s.32xlarge', 'nodeCount': 2}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'TAQ_2021_2D'}, {'key': 'codebase', 'value': 'taqcode'}]",Created with create_cluster_TAQ_2D notebook,2023-06-06 12:54:08.985000+00:00,2023-06-06 12:40:07.108000+00:00,TAQ_2021_2D,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/2021.01.04/', '/2021.01.05/']}]"
HDB_basictickdb_20230601,RUNNING,HDB,"{'nodeType': 'kx.s.xlarge', 'nodeCount': 2}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'basictickdb'}, {'key': 'codebase', 'value': 'basictick'}]",Created with create_HDB for basic_tick notebook,2023-06-06 11:52:52.544000+00:00,2023-06-06 11:35:14.234000+00:00,basictickdb,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"
HDB_welcomedb,RUNNING,HDB,"{'nodeType': 'kx.s.2xlarge', 'nodeCount': 3}","[{'key': 's', 'value': '4'}, {'key': 'dbname', 'value': 'welcomedb'}, {'key': 'codebase', 'value': 'code'}]",Created with create_cluster_HDB notebook,2023-06-06 11:50:28.453000+00:00,2023-06-06 11:32:13.700000+00:00,welcomedb,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"
RDB_basictickdb_20230601,RUNNING,RDB,"{'nodeType': 'kx.s.2xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'basictickdb'}, {'key': 'codebase', 'value': 'basictick'}, {'key': 'tphostfile', 'value': 'tickerplant2'}]",Created with create_RDB notebook,2023-06-06 11:54:07.399000+00:00,2023-06-06 11:38:47.020000+00:00,basictickdb,[]
RDB_welcomedb,RUNNING,RDB,"{'nodeType': 'kx.s.2xlarge', 'nodeCount': 1}","[{'key': 's', 'value': '8'}, {'key': 'dbname', 'value': 'welcomedb'}, {'key': 'codebase', 'value': 'code'}]",Created with create_cluster_RDB notebook,2023-06-06 11:47:08.448000+00:00,2023-06-06 11:32:06.677000+00:00,welcomedb,[]
cluster_create_delete_db_20230606_1132,RUNNING,HDB,"{'nodeType': 'kx.s.xlarge', 'nodeCount': 2}","[{'key': 's', 'value': '4'}, {'key': 'dbname', 'value': 'create_delete_db_20230606_1132'}, {'key': 'codebase', 'value': 'code'}]",Demo Cluster for database create_delete_db_20230606_1132,2023-06-06 11:50:54.875000+00:00,2023-06-06 11:33:18.188000+00:00,create_delete_db_20230606_1132,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"
hdb-cluster-welcomedb,RUNNING,HDB,"{'nodeType': 'kx.s.xlarge', 'nodeCount': 2}","[{'key': 's', 'value': '4'}, {'key': 'dbname', 'value': 'welcomedb'}, {'key': 'codebase', 'value': 'code'}]",None,2023-06-06 11:44:19.914000+00:00,2023-06-06 11:26:51.517000+00:00,welcomedb,"[{'cacheType': 'CACHE_1000', 'dbPaths': ['/']}]"


In [11]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2023-06-06 13:02:08.668157
